In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer,DataCollatorWithPadding

In [3]:
raw_dataset = load_dataset("glue","mrpc")
checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [6]:
#fucntion for tokinizing our imported data
def tokenize_function(ex):
    return tokenizer(ex['sentence1'],ex['sentence2'],truncation=True)

In [7]:
#now doing pre-processing
tokenized_dataset = raw_dataset.map(tokenize_function,batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(['sentence1','sentence2','idx'])
tokenized_dataset = tokenized_dataset.rename_column("label","labels")
tokenized_dataset.set_format("torch")

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [8]:
#using dynamic padding concept
data_collator = DataCollatorWithPadding(tokenizer)

In [9]:
# nwo we need to define the pytorch data loader
from torch.utils.data import DataLoader

In [11]:
#here we are creating our data loaders
train_dataloader = DataLoader(
    tokenized_dataset['train'],shuffle=True,batch_size=8,collate_fn=data_collator)
eval_dataloader = DataLoader(
    tokenized_dataset['validation'],batch_size=8,collate_fn=data_collator)

In [13]:
for batch in train_dataloader:
    break
print({k:v.shape for k,v in batch.items()})

{'labels': torch.Size([8]), 'input_ids': torch.Size([8, 66]), 'token_type_ids': torch.Size([8, 66]), 'attention_mask': torch.Size([8, 66])}


In [14]:
# now we need to create a model
from transformers import AutoModelForSequenceClassification

In [15]:
checkpoint = "bert-base-cased"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint,num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
outputs = model(**batch)
print(outputs.loss,outputs.logits.shape)

tensor(0.6511, grad_fn=<NllLossBackward0>) torch.Size([8, 2])


In [17]:
# now the traing step is happening
# the optimizer will be responsivle for doing the training updates to the model weights

In [18]:
from transformers import AdamW

In [ ]:
optimizer = AdamW(model.parameters(),lr=5e-5)

In [ ]:
loss = outputs.loss
loss.backward()
optimizer.step()

#don't forget to zero our gradients once our optimizer step is done:
optimizer.zero_grad()

In [19]:
# now for betting traing again we are going to introdce learing rate schedular
# A learning rate schedular will update the optimizer's learining rate at each step


In [20]:
from transformers import get_scheduler

In [ ]:
num_epochs = 3
num_traning_steps = num_epochs * len(train_dataset)
lr_schedular = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_training_steps=0,
    num_warmup_steps=num_traning_steps
)

In [21]:
# now for taring we need gpu
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
#model.to(device)
print(device)

cpu


# now putting everythiing together and here is the training loop looks like

In [22]:
from tqdm.auto import tqdm

In [ ]:
progress_bar = tqdm(range(num_training_setps))

In [ ]:
model.train()
for each in range(num_epochs):
    for batch in train_dataloader:
        batch ={k:v.to(device) for k,v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_schedualr.step()
        optimizer.zero_grad()
        progress_bar.update()

In [23]:
# mow compute metrics
from datasets import load_metric

In [ ]:
metric=load_metric("glue","mrpc")
model.eval()
for batch in eval_dataloader:
    batch ={k:v.to(device) for k,v in batch.items()}
    with torch.no_grad():
        outputs=model(**batch)

    logits=outputs.logits
    predictions=torch.argmax(logits,dim=-1)
    metric.add_batch(predictions=predictions,references=batch['labels']
metric.compute()
# now this will show the accurcy and the f1 score

# Supercharge our training loop with Accelerate
we are going to use TrainerAPi for this purpose :- The TrainerApi can handle all those setups for you, but you lose control over your training loop.

In [24]:
from accelerate import Accelerator

In [25]:
accelerator = Accelerator()

In [ ]:
model,optimizer,trai_dataloader=accelerator.prepare(
    model,optimizer,train_dataloader)

In [ ]:
model.train()
for epochs in range(num_apochs):
    outputs=model(**batch)
    loss=outputs.loss
    accelerator.backward(loss)

    optimizer.step()
    lr_schedualr.step()
    optimizer.zero_grad()
    progress_bar.update()
    

In [ ]:
#accelerate also handles distributed evaluation
metric=load_metric("glue","mrpc")
model.eval()

eval_dataloader=accelerator.prepare(eval_dataloader)
for batch in eval_dataloader:
    #batch ={k:v.to(device) for k,v in batch.items()}
    with torch.no_grad():
        outputs=model(**batch)

    logits=outputs.logits
    predictions=torch.argmax(logits,dim=-1)
    #metric.add_batch(predictions=predictions,references=batch['labels']
    metric.add_batch(
        predictions=accelerator.gather(predictions),references=accelerator.gather(batch['labels'])
metric.compute()